In [1]:
from jisho_api.kanji import Kanji
from jisho_api.word import Word
import pandas as pd
from collections import Counter

In [2]:
r = Word.request('りっぱ')

In [3]:
k=r.data

print(r)

meta=RequestMeta(status=200) data=[WordConfig(slug='立派', is_common=True, tags=['wanikani21'], jlpt=['jlpt-n3', 'jlpt-n5'], japanese=[Japanese(word='立派', reading='りっぱ')], senses=[Sense(english_definitions=['splendid', 'fine', 'handsome', 'elegant', 'imposing', 'prominent', 'impressive'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], source=[], info=[]), Sense(english_definitions=['praiseworthy', 'creditable', 'worthy'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], source=[], info=[]), Sense(english_definitions=['legal', 'legitimate', 'lawful'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], source=[], info=[]), Sense(english_definitions=['undeniable (e.g. crime)', 'indisputable', 'solid'], parts_of_speech=['Na-adjective (keiyodoshi)'], links=[], tags=[], restrictions=[], see_also=[], antonyms=[], sour

In [4]:
print(k[0].japanese[0].reading)

りっぱ


In [5]:
r = Kanji.request('立派')

In [6]:
print(r)

meta=RequestMeta(status=200) data=KanjiConfig(kanji='立派', strokes=5, main_meanings=['stand up', 'rise', 'set up', 'erect'], main_readings=MainReadings(kun=['た.つ', '-た.つ', 'た.ち-', 'た.てる', '-た.てる', 'た.て-', 'たて-', '-た.て', '-だ.て', '-だ.てる'], on=['リツ', 'リュウ', 'リットル']), meta=KanjiMeta(education=KanjiMetaEducation(grade=None, jlpt=None, newspaper_rank=None), dictionary_idxs={'149': 'A Guide To Reading and Writing Japanese (Florence Sakade)', '37': 'A Guide To Reading and Writing Japanese 3rd edition (Henshall, Seeley and De Groot)', '73': 'A Guide To Remembering Japanese Characters (Kenneth G. Henshall)', '61': 'A New Dictionary of Kanji Usage', '3343': 'Classic Nelson (Andrew Nelson)', '72': 'Essential Kanji (P.G. O’Neill)', '78': 'Japanese Kanji Flashcards (Max Hodges and Tomoko Okazaki)', '194': 'Japanese Names (P.G. O’Neill)', '2.7': 'Japanese for Busy People', '121': 'Kanji and Kana, 2nd Edition (Spahn and Hadamitzky)', '174': 'Kanji in Context (Nishiguchi and Kono)', '1459': 'Kodansha Co

In [7]:
k = r.data

print("Kanji:", k.kanji)
print("Strokes:", k.strokes)
print("Meanings:", k.main_meanings)
print("Kun readings:", k.main_readings.kun)
print("On readings:", k.main_readings.on)
# From metadata
if k.meta.education:
    print("Grade:", k.meta.education.grade)
    print("JLPT:", k.meta.education.jlpt)
    print("Newspaper rank:", k.meta.education.newspaper_rank)
# Radical info
print("Radical meaning:", k.radical.meaning)
print("Radical basis:", k.radical.basis)
# Reading examples
if k.reading_examples:
    if k.reading_examples.kun:
        print("\nKun Reading Examples:")
        for ex in k.reading_examples.kun:
            print(f"{ex.kanji} ({ex.reading}) → {ex.meanings}")
    if k.reading_examples.on:
        print("\nOn Reading Examples:")
        for ex in k.reading_examples.on:
            print(f"{ex.kanji} ({ex.reading}) → {ex.meanings}")


Kanji: 立派
Strokes: 5
Meanings: ['stand up', 'rise', 'set up', 'erect']
Kun readings: ['た.つ', '-た.つ', 'た.ち-', 'た.てる', '-た.てる', 'た.て-', 'たて-', '-た.て', '-だ.て', '-だ.てる']
On readings: ['リツ', 'リュウ', 'リットル']
Grade: None
JLPT: None
Newspaper rank: None
Radical meaning: stand,
Radical basis: 立

Kun Reading Examples:
立つ (たつ) → ['to stand (up)', 'to rise', "to get to one's feet", 'to stand on end (e.g. of hairs)', 'to stick up', 'to stand (in a position; of a person', 'tree', 'building', 'etc.)', 'to be situated (in', 'on)', 'to be (in difficulties', 'the lead', 'etc.)', 'to put oneself (in a position or situation)', 'to take up (a position', 'post', 'etc.)', 'to position oneself', 'to leave (to do something)', 'to depart (on a journey', 'trip', 'etc.)', 'to leave', 'to set off', 'to start', 'to get stuck (into; of an arrow', 'thorn etc.)', 'to pierce', 'to develop (of a haze', 'waves', 'etc.)', 'to form (of steam', 'bubbles', 'etc.)', 'to appear (of a rainbow', 'clouds', 'etc.)', 'to rise (of sm

In [8]:
def is_kanji(char):
    return (
        '\u4e00' <= char <= '\u9faf' or  # CJK Unified Ideographs
        '\u3400' <= char <= '\u4dbf'    # CJK Unified Ideographs Extension A
    )

In [9]:
jlpt_df=pd.read_csv('dataset/jlpt_vocab.csv').rename(columns={
    'Original':'word',
    'Furigana':'furigana',
    'English':'translation',
    'JLPT Level':'jlpt_level'
})
jlpt_df.head()

,word,furigana,translation,jlpt_level
0,現像,げんぞう,developing (film),N1
1,原則,げんそく,"principle, general rule",N1
2,見地,けんち,point of view,N1
3,現地,げんち,"actual place, local",N1
4,限定,げんてい,"limit, restriction",N1


In [10]:
kanji_list = jlpt_df["word"].apply(
    lambda w: [c for c in w if is_kanji(c)]
)


In [11]:
kanji_list

0       [現, 像]
1       [原, 則]
2       [見, 地]
3       [現, 地]
4       [限, 定]
         ...  
8125       [私]
8126       [私]
8127       [渡]
8128       [渡]
8129       [悪]
Name: word, Length: 8130, dtype: object

In [12]:
all_kanji = [kanji for sublist in kanji_list for kanji in sublist]

In [13]:
kanji_counts = Counter(all_kanji)

In [14]:
kanji_df = pd.DataFrame.from_dict(kanji_counts, orient="index", columns=["count"]).reset_index().rename(columns={'index':'kanji'})

In [15]:
kanji_df.head()

,kanji,count
0,現,19
1,像,5
2,原,20
3,則,4
4,見,50


In [16]:
kana_df=pd.read_csv('dataset/kana_romaji.csv')

In [17]:
kana_map = {}

for _, row in kana_df.iterrows():
    kana_map[row["kana"]] = row["romaji"]


In [18]:
digraphs = {k: v for k, v in kana_map.items() if len(k) == 2}
monographs = {k: v for k, v in kana_map.items() if len(k) == 1}


In [19]:
def kana_to_romaji(text):
    if not isinstance(text, str):
        return None
    
    i = 0
    result = []
    length = len(text)

    while i < length:
        # Handle small tsu: っ or ッ
        if text[i] in ["っ", "ッ"]:
            # Look ahead to next kana (digraph or monograph)
            if i + 2 <= length and text[i+1:i+3] in digraphs:
                # double first consonant of romaji
                next_romaji = digraphs[text[i+1:i+3]]
                result.append(next_romaji[0])
                i += 1
                continue
            elif i + 1 < length and text[i+1] in monographs:
                next_romaji = monographs[text[i+1]]
                result.append(next_romaji[0])
                i += 1
                continue
            else:
                # Lone small tsu — rare — ignore
                i += 1
                continue

        # Digraph (e.g. ちゃ, きゃ)
        if i + 2 <= length and text[i:i+2] in digraphs:
            result.append(digraphs[text[i:i+2]])
            i += 2
            continue

        # Monograph
        if text[i] in monographs:
            result.append(monographs[text[i]])
            i += 1
            continue

        # Unknown character (kanji or punctuation) → keep as-is or skip
        result.append(text[i])
        i += 1

    return "".join(result)


In [ ]:
jlpt_df["romaji"] = jlpt_df["furigana"].apply(kana_to_romaji)

In [23]:
jlpt_df["num_characters"] = jlpt_df["word"].apply(lambda x: len(str(x)))


In [24]:
jlpt_df.head()

,word,furigana,translation,jlpt_level,romaji,num_characters
0,現像,げんぞう,developing (film),N1,genzou,2
1,原則,げんそく,"principle, general rule",N1,gensoku,2
2,見地,けんち,point of view,N1,kenchi,2
3,現地,げんち,"actual place, local",N1,genchi,2
4,限定,げんてい,"limit, restriction",N1,gentei,2


In [ ]:
result = Kanji.request("品")  # example kanji
print(result.data.kanji)
print(result.data.main_readings.kun)
print(result.data.main_readings.on)
print(result.data.strokes)


品
しな
ヒン
9


In [25]:
import time

def get_kanji_strokes(k):
    time.sleep(0.1)
    try:
        r = Kanji.request(k)
        return r.data.strokes
    except:
        return None

def get_kanji_kun_readings(k):
    time.sleep(0.1)
    try:
        r = Kanji.request(k)
        return r.data.main_readings.kun
    except:
        return None
    
def get_kanji_on_readings(k):
    time.sleep(0.1)
    try:
        r = Kanji.request(k)
        return r.data.main_readings.on
    except:
        return None

In [26]:
kanji_df['strokes'] = kanji_df['kanji'].apply(get_kanji_strokes)

In [27]:
kanji_df['kun_readings'] = kanji_df['kanji'].apply(get_kanji_kun_readings)

In [28]:
kanji_df['on_readings'] = kanji_df['kanji'].apply(get_kanji_on_readings)

In [29]:
kanji_df.head()

,kanji,count,strokes,kun_readings,on_readings
0,現,19,11,"[あらわ.れる, あらわ.す, うつつ, うつ.つ]",[ゲン]
1,像,5,14,None,[ゾウ]
2,原,20,10,[はら],[ゲン]
3,則,4,9,"[のっと.る, のり, すなわち]",[ソク]
4,見,50,7,"[み.る, み.える, み.せる]",[ケン]


In [30]:
kanji_df.to_csv('dataset/kanji.csv', index=False)